In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import zscore
from scipy.stats import stats
from scipy.stats import linregress
import os
import seaborn as sns
import re
import datetime
import nbconvert
import glob
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf
import chart_studio

from statsmodels.compat import lzip
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
# Files to Load (csv files downloaded from the internet)

# Country information files
file1 = "data/unsd.csv"
file2 = "data/six-dem_mod.csv"

# Covid Case timeseries file
file3 = "data/covid-confirmed-cases-since-100th-case_hkg.csv"
file4 = "data/worldometer.csv"

# Country timeseries files
file5 = "data/population-density-vs-prosperity.csv"
file6 = "data/urban-and-rural-population.csv"
file7 = "data/urban-vs-rural-majority.csv"
file8 = "data/life-expectancy-at-birth-total-years.csv"
file9 = "data/median-age.csv"

# Read File and store into Pandas data frame
unsd = pd.read_csv(file1)
hoefstede = pd.read_csv(file2)

covid_data = pd.read_csv(file3, parse_dates=['Date'])
wom = pd.read_csv(file4,parse_dates=['date'])

density_vs_prosperity = pd.read_csv(file5)
urban_rural_population = pd.read_csv(file6)
urban_vs_rural_majority = pd.read_csv(file7)
life_expectancy = pd.read_csv(file8)
median_age = pd.read_csv(file9)


# Create 'countries' dataframe with all country information
#### This is in Tidy format

In [3]:
unsd.columns

Index(['Global Code', 'Global Name', 'Region Code', 'Region Name',
       'Sub-region Code', 'Sub-region Name', 'Intermediate Region Code',
       'Intermediate Region Name', 'Country or Area', 'M49 Code', 'Code',
       'Least Developed Countries (LDC)',
       'Land Locked Developing Countries (LLDC)',
       'Small Island Developing States (SIDS)',
       'Developed / Developing Countries'],
      dtype='object')

In [4]:
#Format columns
unsd = unsd.rename(columns={"Global Code":"globalCode",
                            "Global Name":"globalName",
                            'Region Code':"regionCode",
                            'Region Name':"regionName",
                            'Sub-region Code':"subRegionCode",
                            'Sub-region Name':"subRegionName",
                            'Intermediate Region Code':"interRegionCode",
                            'Intermediate Region Name':"interRegionName",
                            'Country or Area':"country",
                            'M49 Code':"m49",
                            'Code':"code",
                            'Least Developed Countries (LDC)':"ldc",
                            'Land Locked Developing Countries (LLDC)':"lldc",
                            'Small Island Developing States (SIDS)':"sids",
                            'Developed / Developing Countries':"development"
                            })

In [5]:
hoefstede.columns

Index(['ctr', 'country', 'pdi', 'idv', 'mas', 'uai', 'ltowvs', 'ivr'], dtype='object')

In [6]:
#Format columns
hoefstede = hoefstede.rename(columns={"ctr":"code"
                            })

In [7]:
hoefstede = hoefstede.replace('#NULL!','')

In [8]:
hoefstede[['pdi','idv','mas','uai','ltowvs','ivr']] = hoefstede[['pdi','idv','mas','uai','ltowvs','ivr']].apply(pd.to_numeric)

In [9]:
hoefstede.head()

,code,country,pdi,idv,mas,uai,ltowvs,ivr
0,GTM,Guatemala,95.0,6.0,37.0,101.0,NaN,NaN
1,ECU,Ecuador,78.0,8.0,63.0,67.0,NaN,NaN
2,PAN,Panama,95.0,11.0,44.0,86.0,NaN,NaN
3,VEN,Venezuela,81.0,12.0,73.0,76.0,16.0,100.0
4,COL,Colombia,67.0,13.0,64.0,80.0,13.0,83.0


In [10]:
# Merge data
countries = pd.merge(unsd, hoefstede, on=["code"])
countries.shape

(118, 22)

In [11]:
countries.columns

Index(['globalCode', 'globalName', 'regionCode', 'regionName', 'subRegionCode',
       'subRegionName', 'interRegionCode', 'interRegionName', 'country_x',
       'm49', 'code', 'ldc', 'lldc', 'sids', 'development', 'country_y', 'pdi',
       'idv', 'mas', 'uai', 'ltowvs', 'ivr'],
      dtype='object')

In [12]:
countries.drop(['country_y'], axis=1,inplace=True)

In [13]:
countries = countries.rename(columns={'country_x': 'country'})

In [14]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 0 to 117
Data columns (total 21 columns):
globalCode         118 non-null int64
globalName         118 non-null object
regionCode         118 non-null int64
regionName         118 non-null object
subRegionCode      118 non-null int64
subRegionName      118 non-null object
interRegionCode    38 non-null float64
interRegionName    36 non-null object
country            118 non-null object
m49                118 non-null int64
code               118 non-null object
ldc                17 non-null float64
lldc               15 non-null float64
sids               7 non-null float64
development        118 non-null object
pdi                102 non-null float64
idv                102 non-null float64
mas                102 non-null float64
uai                102 non-null float64
ltowvs             97 non-null float64
ivr                95 non-null float64
dtypes: float64(10), int64(4), object(7)
memory usage: 20.3+ KB


In [15]:
# Export CSV
countries.to_csv('countries.csv')  

In [16]:
# Create multiindex using unsd naming conventions
countries = countries.set_index(['globalName','regionName','subRegionName','code'])

In [17]:
countries.head()

globalCode  \
globalName regionName subRegionName                   code               
World      Europe     Southern Europe                 ALB            1   
           Africa     Northern Africa                 DZA            1   
           Europe     Southern Europe                 AND            1   
           Africa     Sub-Saharan Africa              AGO            1   
           Americas   Latin America and the Caribbean ARG            1   

                                                            regionCode  \
globalName regionName subRegionName                   code               
World      Europe     Southern Europe                 ALB          150   
           Africa     Northern Africa                 DZA            2   
           Europe     Southern Europe                 AND          150   
           Africa     Sub-Saharan Africa              AGO            2   
           Americas   Latin America and the Caribbean ARG           19   

                                                            subRegionCode  \
globalName regionName subRegionName                   code                  
World      Europe     Southern Europe                 ALB              39   
           Africa     Northern Africa                 DZA              15   
           Europe     Southern Europe                 AND              39   
           Africa     Sub-Saharan Africa              AGO             202   
           Americas   Latin America and the Caribbean ARG             419   

                                                            interRegionCode  \
globalName regionName subRegionName                   code                    
World      Europe     Southern Europe                 ALB               NaN   
           Africa     Northern Africa                 DZA               NaN   
           Europe     Southern Europe                 AND               NaN   
           Africa     Sub-Saharan Africa              AGO              17.0   
           Americas   Latin America and the Caribbean ARG               5.0   

                                                           interRegionName  \
globalName regionName subRegionName                   code                   
World      Europe     Southern Europe                 ALB              NaN   
           Africa     Northern Africa                 DZA              NaN   
           Europe     Southern Europe                 AND              NaN   
           Africa     Sub-Saharan Africa              AGO    Middle Africa   
           Americas   Latin America and the Caribbean ARG    South America   

                                                              country  m49  \
globalName regionName subRegionName                   code                   
World      Europe     Southern Europe                 ALB     Albania    8   
           Africa     Northern Africa                 DZA     Algeria   12   
           Europe     Southern Europe                 AND     Andorra   20   
           Africa     Sub-Saharan Africa              AGO      Angola   24   
           Americas   Latin America and the Caribbean ARG   Argentina   32   

                                                            ldc  lldc  sids  \
globalName regionName subRegionName                   code                    
World      Europe     Southern Europe                 ALB   NaN   NaN   NaN   
           Africa     Northern Africa                 DZA   NaN   NaN   NaN   
           Europe     Southern Europe                 AND   NaN   NaN   NaN   
           Africa     Sub-Saharan Africa              AGO   1.0   NaN   NaN   
           Americas   Latin America and the Caribbean ARG   NaN   NaN   NaN   

                                                           development   pdi  \
globalName regionName subRegionName                   code                     
World      Europe     Southern Europe                 ALB    Developed  90.0   
           Africa     Northern Africa   

# Create 'covid' dataframe with timeseries & confirmed cases

In [18]:
covid_data.columns

Index(['Entity', 'Code', 'Date', ' (cases)',
       'Number of days since the 100th confirmed case (days)'],
      dtype='object')

In [19]:
covid_data.head()

,Entity,Code,Date,(cases),Number of days since the 100th confirmed case (days)
0,Afghanistan,AFG,2019-12-31,0.0,NaN
1,Afghanistan,AFG,2020-01-01,0.0,NaN
2,Afghanistan,AFG,2020-01-02,0.0,NaN
3,Afghanistan,AFG,2020-01-03,0.0,NaN
4,Afghanistan,AFG,2020-01-04,0.0,NaN


In [20]:
#Format columns
covid = covid_data.rename(columns={'Entity': "country",
                              'Code' : "code",
                              'Date': "date",
                              ' (cases)' : "cases",
                              'Number of days since the 100th confirmed case (days)':"daysGT100"
                                         })
covid = covid[['code','date','cases','daysGT100']]

In [21]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12768 entries, 0 to 12767
Data columns (total 4 columns):
code         11092 non-null object
date         12768 non-null datetime64[ns]
cases        12761 non-null float64
daysGT100    4459 non-null float64
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 399.1+ KB


In [22]:
density_vs_prosperity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47234 entries, 0 to 47233
Data columns (total 6 columns):
Entity                                              47234 non-null object
Code                                                44392 non-null object
Year                                                47234 non-null int64
Population density (people per km² of land area)    14600 non-null float64
GDP per capita (constant 2011 international $)      6407 non-null float64
Total population (Gapminder)                        43340 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 2.2+ MB


In [23]:
density_vs_prosperity.dropna(subset=['Code'],inplace=True)
density_vs_prosperity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44392 entries, 0 to 47233
Data columns (total 6 columns):
Entity                                              44392 non-null object
Code                                                44392 non-null object
Year                                                44392 non-null int64
Population density (people per km² of land area)    11978 non-null float64
GDP per capita (constant 2011 international $)      5167 non-null float64
Total population (Gapminder)                        43120 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 2.4+ MB


In [24]:
urban_rural_population.dropna(subset=['Code'],inplace=True)
urban_rural_population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12363 entries, 0 to 15030
Data columns (total 5 columns):
Entity              12363 non-null object
Code                12363 non-null object
Year                12363 non-null int64
Urban population    12363 non-null int64
Rural population    12363 non-null int64
dtypes: int64(3), object(2)
memory usage: 579.5+ KB


In [25]:
urban_vs_rural_majority.dropna(subset=['Code'],inplace=True)
urban_vs_rural_majority.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24019 entries, 0 to 27976
Data columns (total 5 columns):
Entity       24019 non-null object
Code         24019 non-null object
Year         24019 non-null int64
Urban (%)    24019 non-null float64
Rural (%)    24019 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.1+ MB


In [26]:
#urban_vs_rural_majority['Year'] = pd.to_datetime(urban_vs_rural_majority['Year'])

In [27]:
life_expectancy.dropna(subset=['Code'],inplace=True)
life_expectancy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11125 entries, 0 to 13746
Data columns (total 4 columns):
Entity                                             11125 non-null object
Code                                               11125 non-null object
Year                                               11125 non-null int64
Life expectancy at birth, total (years) (years)    11125 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 434.6+ KB


In [28]:
median_age.dropna(subset=['Code'],inplace=True)
median_age.shape

(6324, 4)

In [29]:
covid_data.shape

(12768, 5)

In [30]:
# Merge data
merged_data = pd.merge(density_vs_prosperity, urban_rural_population, on=["Entity","Code", "Year"])
merged_data.shape

(12283, 8)

In [31]:
merged_data.head()

,Entity,Code,Year,Population density (people per km² of land area),GDP per capita (constant 2011 international $),Total population (Gapminder),Urban population,Rural population
0,Afghanistan,AFG,1960,NaN,NaN,8996351.0,755783,8240568
1,Afghanistan,AFG,1961,14.040934,NaN,9166764.0,796042,8370722
2,Afghanistan,AFG,1962,14.315271,NaN,9345868.0,838885,8506983
3,Afghanistan,AFG,1963,14.603367,NaN,9533954.0,884370,8649584
4,Afghanistan,AFG,1964,14.905739,NaN,9731361.0,932848,8798513


In [32]:
urban_vs_rural_majority.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24019 entries, 0 to 27976
Data columns (total 5 columns):
Entity       24019 non-null object
Code         24019 non-null object
Year         24019 non-null int64
Urban (%)    24019 non-null float64
Rural (%)    24019 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.1+ MB


In [33]:
merged_data = pd.merge(merged_data, urban_vs_rural_majority, on=["Entity", "Code", "Year"])
merged_data.shape

(12283, 10)

In [34]:
merged_data = pd.merge(merged_data, life_expectancy, on=["Entity", "Code", "Year"])
merged_data.shape

(11007, 11)

In [38]:
merged_data.columns

Index(['Entity', 'Code', 'Year',
       'Population density (people per km² of land area)',
       'GDP per capita (constant 2011 international $)',
       'Total population (Gapminder)', 'Urban population', 'Rural population',
       'Urban (%)', 'Rural (%)',
       'Life expectancy at birth, total (years) (years)'],
      dtype='object')

In [35]:
merged_data = pd.merge(merged_data, unsd, on=["Code"])
merged_data.shape

KeyError: 'Code'

In [ ]:
merged_data = pd.merge(merged_data, wom, on=["Code"])
merged_data.shape

In [ ]:
merged_data.head()

In [ ]:
merged_data = pd.merge(merged_data, median_age, on=["Entity", "Code", "Year"])
merged_data.shape

In [ ]:
merged_data.head()

In [ ]:
merged_data.columns

In [ ]:
#Format columns
merged_data = merged_data.rename(columns={"Population density (people per km² of land area)": "popDensity",
                                          "GDP per capita (constant 2011 international $)" : "gdp",
                                         "Total population (Gapminder)": "gapPop",
                                         "Urban population": "urbanPop",
                                         "Rural population": "ruralPop",
                                         "Urban (%)":"urbanPct",
                                         "Rural (%)": "ruralPct",
                                         "Life expectancy at birth, total (years) (years)":"lifeExpectancy",
                                         "UN Population Division (Median Age) (2017) (years)":"medianAge",
                                         "Global Code":"globalCode",
                                         "Global Name":"globalName",
                                         'Region Code':"regionCode",
                                          'Region Name':"regionName",
                                          'Sub-region Code':"subRegionCode",
                                          'Sub-region Name':"subRegionName",
                                          'Intermediate Region Code':"interRegionCode",
                                          'Intermediate Region Name':"interRegionName",
                                          'Country or Area':"country",
                                          'M49 Code':"m49",
                                          'Least Developed Countries (LDC)':"ldc",
                                          'Land Locked Developing Countries (LLDC)':"lldc",
                                          'Small Island Developing States (SIDS)':"sids",
                                          'Developed / Developing Countries':"development",
                                          'UN Population Division (Median Age) (2017) (years)_y': 'medianAge'
                                         })

In [ ]:
merged_data.columns

In [ ]:
merged_data.head()

In [ ]:
hoefstede.shape

In [ ]:
#hoefstede.head()

In [ ]:
hoefstede = hoefstede.rename(columns={"ctr": "Code",
                                          "country" : "Entity"})
#hoefstede.head()

In [ ]:
covid_data.shape

In [ ]:
merged_data = pd.merge(merged_data, hoefstede, on=[ "Code"])
#merged_data.head()

In [ ]:
merged_data.shape

In [ ]:
merged_data['Year'].max()

In [ ]:
merged_data = merged_data[merged_data["Year"] == 2015]
#merged_data.head()

In [ ]:
merged_data.drop(['Entity_y','Year'], axis=1,inplace=True)

In [ ]:
merged_data = merged_data.rename(columns={"Entity_x": "Entity"})

In [ ]:
merged_data.head()

In [ ]:
merged_data.shape

In [ ]:
#covid_data.head()

In [ ]:
merged_data = pd.merge(merged_data, covid_data, on=[ "Code"])
#merged_data.head()

In [ ]:
merged_data.drop(['Entity_y'], axis=1,inplace=True)

In [ ]:
merged_data = merged_data.rename(columns={"Entity_x": "Entity",
                                         " (cases)" : "cases",
                                          "Number of days since the 100th confirmed case (days)":"days_gt_100"})

In [ ]:
merged_data.shape

In [ ]:
merged_data.tail()

In [ ]:
# Drop rows where days_gt_100 is null
#merged_data = merged_data[merged_data.days_gt_100 != 0]
merged_data.dropna(subset=['days_gt_100'],inplace=True)
merged_data.shape

In [ ]:
merged_data['log_cases'] = np.log(merged_data['cases'])
merged_data['casesPop'] = (merged_data['cases']/merged_data['calcPop'])
merged_data['logcasesPop'] = np.log(merged_data['cases']/merged_data['calcPop'])
merged_data['logcasesMPop'] = np.log(merged_data['casesMPop'])
# merged_data['log_cases'] = np.log(merged_data['cases'])
# merged_data['log_cases'] = np.log(merged_data['cases'])

In [ ]:
merged_data.info()

In [ ]:
# Export CSV
merged_data.to_csv('merged_data_modified.csv')  

In [ ]:
merged_data['Date'] = pd.to_timedelta(merged_data['Date']).dt.total_seconds().astype(int)
#merged_data['Date'] = pd.to_datetime(merged_data['Date'], format='%Y-%m-%d').dt.time

In [ ]:
#merged_data.head()

## Create subset of Countries that have reached 5 days > 100 cases

In [ ]:
cases5 = merged_data[merged_data.groupby('Code')['days_gt_100'].transform(lambda x: (x == 5).sum())>0]

In [ ]:
cases5group = cases5.groupby('Code')

In [ ]:
cases5slopes = (cases5group.apply(lambda x: pd.Series(linregress(x['Date'],x['log_cases'])))\
        .rename(columns={
        0: 'slope',
        1: 'intercept',
        2: 'rvalue',
        3: 'pvalue',
        4: 'stderr'
    }))
casesPop5slopes = (cases5group.apply(lambda x: pd.Series(linregress(x['Date'],x['logcasesPop'])))\
        .rename(columns={
        0: 'slopePop',
        1: 'interceptPop',
        2: 'rvaluePop',
        3: 'pvaluePop',
        4: 'stderrPop'
    }))

In [ ]:
merged5 = merged_data[merged_data.days_gt_100 == 5]

In [ ]:
mergedcases5 = pd.merge(merged5, cases5slopes, on=["Code"])
mergedcases5 = pd.merge(mergedcases5, casesPop5slopes, on=["Code"])

In [ ]:
#mergedcases5.dropna(subset=['idv'],inplace=True)

In [ ]:
# Export CSV
mergedcases5.to_csv('cases5.csv')  

In [ ]:
corr5 = mergedcases5.corr()

In [ ]:
# Plot the Pearson Correlation reduced matrix
matrix = np.triu(corr5)
fig, ax = plt.subplots(figsize=(15,9)) 
ax = sns.heatmap(corr5,   mask = matrix, cmap = 'RdYlBu')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title('Pearson Correlation matrix for Countries at 5 days since 100th Covid case')

In [ ]:
corr5.to_csv('corrcases5.csv')

## Create subset of Countries that have reached 10 days > 100 cases

In [ ]:
cases10 = merged_data[merged_data.groupby('Code')['days_gt_100'].transform(lambda x: (x == 10).sum())>0]

In [ ]:
cases10group = cases10.groupby('Code')

In [ ]:
cases10slopes = (cases10group.apply(lambda x: pd.Series(linregress(x['Date'],x['log_cases'])))\
        .rename(columns={
        0: 'slope',
        1: 'intercept',
        2: 'rvalue',
        3: 'pvalue',
        4: 'stderr'
    }))
casesPop10slopes = (cases10group.apply(lambda x: pd.Series(linregress(x['Date'],x['logcasesPop'])))\
        .rename(columns={
        0: 'slopePop',
        1: 'interceptPop',
        2: 'rvaluePop',
        3: 'pvaluePop',
        4: 'stderrPop'
    }))

In [ ]:
merged10 = merged_data[merged_data.days_gt_100 == 10]

In [ ]:
mergedcases10 = pd.merge(merged10, cases10slopes, on=["Code"])
mergedcases10 = pd.merge(mergedcases10, casesPop10slopes, on=["Code"])

In [ ]:
# Export CSV
mergedcases10.to_csv('cases10.csv')  

In [ ]:
corr10 = mergedcases10.corr()

In [ ]:
# Plot the Pearson Correlation reduced matrix
matrix = np.triu(corr10)
fig, ax = plt.subplots(figsize=(15,9)) 
ax = sns.heatmap(corr10,   mask = matrix, cmap = 'RdYlBu')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title('Pearson Correlation matrix for Countries at 10 days since 100th Covid case')

In [ ]:
corr10.to_csv('corrcases10.csv')

## Create subset of Countries that have reached 20 days > 100 cases

In [ ]:
cases20 = merged_data[merged_data.groupby('Code')['days_gt_100'].transform(lambda x: (x == 20).sum())>0]

In [ ]:
cases20group = cases20.groupby('Code')

In [ ]:
cases20slopes = (cases20group.apply(lambda x: pd.Series(linregress(x['Date'],x['log_cases'])))\
        .rename(columns={
        0: 'slope',
        1: 'intercept',
        2: 'rvalue',
        3: 'pvalue',
        4: 'stderr'
    }))
casesPop20slopes = (cases20group.apply(lambda x: pd.Series(linregress(x['Date'],x['logcasesPop'])))\
        .rename(columns={
        0: 'slopePop',
        1: 'interceptPop',
        2: 'rvaluePop',
        3: 'pvaluePop',
        4: 'stderrPop'
    }))

In [ ]:
merged20 = merged_data[merged_data.days_gt_100 == 20]

In [ ]:
mergedcases20 = pd.merge(merged20, cases20slopes, on=["Code"])
mergedcases20 = pd.merge(mergedcases20, casesPop20slopes, on=["Code"])

In [ ]:
# Export CSV
mergedcases20.to_csv('cases20.csv')  

In [ ]:
corr20 = mergedcases20.corr()

In [ ]:
# Plot the Pearson Correlation reduced matrix
matrix = np.triu(corr20)
fig, ax = plt.subplots(figsize=(15,9)) 
ax = sns.heatmap(corr20,   mask = matrix, cmap = 'RdYlBu')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title('Pearson Correlation matrix for Countries at 20 days since 100th Covid case')

In [ ]:
corr20.to_csv('corrcases20.csv')

## Create subset of Countries that have reached 30 days > 100 cases

In [ ]:
cases30 = merged_data[merged_data.groupby('Code')['days_gt_100'].transform(lambda x: (x == 30).sum())>0]

In [ ]:
cases30group = cases30.groupby('Code')

In [ ]:
cases30slopes = (cases30group.apply(lambda x: pd.Series(linregress(x['Date'],x['log_cases'])))\
        .rename(columns={
        0: 'slope',
        1: 'intercept',
        2: 'rvalue',
        3: 'pvalue',
        4: 'stderr'
    }))
casesPop30slopes = (cases30group.apply(lambda x: pd.Series(linregress(x['Date'],x['logcasesPop'])))\
        .rename(columns={
        0: 'slopePop',
        1: 'interceptPop',
        2: 'rvaluePop',
        3: 'pvaluePop',
        4: 'stderrPop'
    }))

In [ ]:
merged30 = merged_data[merged_data.days_gt_100 == 30]

In [ ]:
mergedcases30 = pd.merge(merged30, cases30slopes, on=["Code"])
mergedcases30 = pd.merge(mergedcases30, casesPop30slopes, on=["Code"])

In [ ]:
# Export CSV
mergedcases30.to_csv('cases30.csv')  

In [ ]:
corr30 = mergedcases30.corr()

In [ ]:
# Plot the Pearson Correlation reduced matrix
matrix = np.triu(corr30)
fig, ax = plt.subplots(figsize=(15,9)) 
ax = sns.heatmap(corr30,   mask = matrix, cmap = 'RdYlBu')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title('Pearson Correlation matrix for Countries at 30 days since 100th Covid case')

In [ ]:
corr30.to_csv('corrcases30.csv')

## Create subset of Countries that have reached 35 days > 100 cases

In [ ]:
cases35 = merged_data[merged_data.groupby('Code')['days_gt_100'].transform(lambda x: (x == 35).sum())>0]

In [ ]:
cases35group = cases35.groupby('Code')

In [ ]:
cases35slopes = (cases35group.apply(lambda x: pd.Series(linregress(x['Date'],x['log_cases'])))\
        .rename(columns={
        0: 'slope',
        1: 'intercept',
        2: 'rvalue',
        3: 'pvalue',
        4: 'stderr'
    }))
casesPop35slopes = (cases35group.apply(lambda x: pd.Series(linregress(x['Date'],x['logcasesPop'])))\
        .rename(columns={
        0: 'slopePop',
        1: 'interceptPop',
        2: 'rvaluePop',
        3: 'pvaluePop',
        4: 'stderrPop'
    }))

In [ ]:
merged35 = merged_data[merged_data.days_gt_100 == 35]

In [ ]:
mergedcases35 = pd.merge(merged35, cases35slopes, on=["Code"])
mergedcases35 = pd.merge(mergedcases35, casesPop35slopes, on=["Code"])

In [ ]:
# Export CSV
mergedcases35.to_csv('cases35.csv')  

In [ ]:
corr35 = mergedcases35.corr()

In [ ]:
# Plot the Pearson Correlation reduced matrix
matrix = np.triu(corr35)
fig, ax = plt.subplots(figsize=(15,9)) 
ax = sns.heatmap(corr35,   mask = matrix, cmap = 'RdYlBu')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title('Pearson Correlation matrix for Countries at 30 days since 100th Covid case')

In [ ]:
corr35.to_csv('corrcases35.csv')

## Create subset of Countries that have reached 40 days > 100 cases

In [ ]:
cases40 = merged_data[merged_data.groupby('Code')['days_gt_100'].transform(lambda x: (x == 40).sum())>0]

In [ ]:
cases40group = cases40.groupby('Code')

In [ ]:
cases40slopes = (cases40group.apply(lambda x: pd.Series(linregress(x['Date'],x['log_cases'])))\
        .rename(columns={
        0: 'slope',
        1: 'intercept',
        2: 'rvalue',
        3: 'pvalue',
        4: 'stderr'
    }))
casesPop40slopes = (cases40group.apply(lambda x: pd.Series(linregress(x['Date'],x['logcasesPop'])))\
        .rename(columns={
        0: 'slopePop',
        1: 'interceptPop',
        2: 'rvaluePop',
        3: 'pvaluePop',
        4: 'stderrPop'
    }))

In [ ]:
merged40 = merged_data[merged_data.days_gt_100 == 40]

In [ ]:
mergedcases40 = pd.merge(merged40, cases40slopes, on=["Code"])
mergedcases40 = pd.merge(mergedcases40, casesPop40slopes, on=["Code"])

In [ ]:
# Export CSV
mergedcases40.to_csv('cases40.csv')  

In [ ]:
corr40 = mergedcases40.corr()

In [ ]:
# Plot the Pearson Correlation reduced matrix
matrix = np.triu(corr40)
fig, ax = plt.subplots(figsize=(15,9)) 
ax = sns.heatmap(corr40,   mask = matrix, cmap = 'RdYlBu')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title('Pearson Correlation matrix for Countries at 40 days since 100th Covid case')

In [ ]:
corr40.to_csv('corrcases40.csv')

### Concatonate the dataframes back together into one dataframe

In [ ]:
# Concatonate all the dataframes 
cases_df = pd.concat([mergedcases40, mergedcases35, mergedcases30, mergedcases20, mergedcases10, mergedcases5])

In [ ]:
cases_df.head()

In [ ]:
cases_df.to_csv('cases_df.csv')

In [ ]:
cases_df.columns

In [ ]:
cases_df.head()
multi_slope = cases_df.set_index(['globalName','regionName','subRegionName','country'])
multi_slope = multi_slope.sort_index()
print(multi_slope.shape)
multi_slope.head()

### Function to easily do a panel of scatterplots easily

In [ ]:
def makePanelScatterplot(data_df, x_data,y_data,color_col,split_col):
    x_label = x_data
    y_label = y_data
    ylim_min = data_df[y_data].min()*1.05
    ylim_max = data_df[y_data].max()*1.05
    xlim_min = data_df[x_data].min()*1.05
    xlim_max = data_df[x_data].max()*1.05

    g = sns.lmplot(x=x_data, y=y_data, data=data_df,\
           hue=color_col, col=split_col,col_wrap=3)

        
    g = (g.set_axis_labels(x_label, y_label)
          .set(ylim=(ylim_min, ylim_max),xlim=(xlim_min, xlim_max))
          .fig.subplots_adjust(wspace=.02))

# Slope of # Covid Cases vs. Power-Distance (PDI) of Culture

In [ ]:
makePanelScatterplot(cases_df,"pdi","slopePop","days_gt_100","days_gt_100")

# Slope of # Covid Cases vs. Individualism/Collectivism of Culture

In [ ]:
makePanelScatterplot(cases_df,"ivr","slopePop","days_gt_100","days_gt_100")

# Slope of # Covid Cases vs. Indulgence (IVR) of Culture

In [ ]:
makePanelScatterplot(cases_df,"ivr","slopePop","days_gt_100","days_gt_100")

# Slope of # Covid Cases vs. Long-term Orientation (LTOWVS) of Culture

In [ ]:
makePanelScatterplot(cases_df,"ltowvs","slopePop","days_gt_100","days_gt_100")

# Slope of # Covid Cases vs. Masculine/Feminine (MAS) of Culture

In [ ]:
makePanelScatterplot(cases_df,"mas","slope","days_gt_100","days_gt_100")

# Slope of # Covid Cases vs. Uncertainty Avoidance (UAI) of Culture

In [ ]:
makePanelScatterplot(cases_df,"uai","slope","days_gt_100","days_gt_100")

In [ ]:
cases_df.columns

In [ ]:
makePanelScatterplot(cases_df,"lifeExpectancy","medianAge","days_gt_100","days_gt_100")

# Change is

In [ ]:
makePanelScatterplot(cases_df,"logcasesPop","slopePop","days_gt_100","days_gt_100")

In [ ]:
cases_df.columns

In [ ]:
multi_slope.head()

In [ ]:
makePanelScatterplot(cases_df,"ivr","slopePop","days_gt_100","days_gt_100")

In [ ]:
def olsRegressionAnalysis (df,df_name,dep_col,ind_col):
    stat_dep_col = dep_col.replace(' ','_').replace('-','_').replace('(','_').replace(')','')
    stat_ind_col = ind_col.replace(' ','_').replace('-','_').replace('(','_').replace(')','')
    stat_col_list = [stat_dep_col,stat_ind_col]
    #print(f'stat_dep_col: {stat_dep_col}, stat_ind_col: {stat_ind_col}')
    col_list = [dep_col,ind_col]

    col_dict = {col_list[i]: stat_col_list[i] for i in range(len(col_list))} 
    #print(col_dict)
    stat_df = df[col_list].dropna()
    stat_df.rename(columns=col_dict,inplace=True)
    print(f'\u001b[34m{dep_col}\u001b[0m fitted against \u001b[34m{ind_col}\u001b[0m \
    using \x1b[31m{df_name}\x1b[0m dataframe:\n')
    print(f'We have {stat_df.shape[0]} rows left after dropping Null values\n')
    model_string = stat_dep_col + " ~ " + stat_ind_col
    all_model = ols(model_string, data=stat_df).fit()
    print(all_model.summary())

In [ ]:
cases_df.columns

In [ ]:
# Create tidy datasets for Country then Covid
country_cols = ['Entity', 'Code', 'popDensity', 'gdp', 'gapPop', 'urbanPop', 'ruralPop',
       'urbanPct', 'ruralPct', 'lifeExpectancy', 'globalCode', 'globalName',
       'regionCode', 'regionName', 'subRegionCode', 'subRegionName',
       'interRegionCode', 'interRegionName', 'country', 'm49', 'ldc', 'lldc',
       'sids', 'development', 'countryWom', 'calcPop', 'medianAge', 'pdi',
       'idv', 'mas', 'uai', 'ltowvs', 'ivr']
countries = cases_df[country_cols].drop_duplicates()
countries = countries.reset_index(drop=True)
countries = countries.set_index(['Code'])
countries.head()

In [ ]:
countries.columns

In [ ]:
# Create Covid cases table
covid_vc = pd.merge(cases_df, countries, on=['Entity', 'popDensity', 'gdp', 'gapPop', 'urbanPop', 'ruralPop',
       'urbanPct', 'ruralPct', 'lifeExpectancy', 'globalCode', 'globalName',
       'regionCode', 'regionName', 'subRegionCode', 'subRegionName',
       'interRegionCode', 'interRegionName', 'country', 'm49', 'ldc', 'lldc',
       'sids', 'development', 'countryWom', 'calcPop', 'medianAge', 'pdi',
       'idv', 'mas', 'uai', 'ltowvs', 'ivr'])
covid_vc.columns

In [ ]:
covid_vc = covid_vc[['Code','Date', 'cases', 'days_gt_100',
       'log_cases', 'casesPop', 'logcasesPop', 'logcasesMPop', 'slope',
       'intercept', 'rvalue', 'pvalue', 'stderr', 'slopePop', 'interceptPop',
       'rvaluePop', 'pvaluePop', 'stderrPop']]
covid_vc.head()

In [ ]:
tidy_df = cases_df.melt(id_vars="days_gt_100")

In [ ]:
cases_df.head()

In [ ]:
px.line(cases_df, x='days_gt_100',y='log_cases')

In [ ]:
tidy_df.columns

In [ ]:
import plotly.graph_objects as go
fig = go.FigureWidget(data=go.Bar(y=[2, 3, 1]))
fig.write_image('figure.png')

In [ ]:
merged_data = px.data.gapminder().query("continent != 'Asia'") # remove Asia for visibility
fig = px.line(df, x="year", y="lifeExp", color="continent",
              line_group="country", hover_name="country")
fig.show()

In [ ]:
import plotly.express as px
df = cases_df.query("rangeName == 'Asia'")
fig = px.line(df, x='days_gt_100', y='slope', color='country')
fig.show()

In [ ]:
#olsRegressionAnalysis(cases40,"log_cases","slope","Code")

In [ ]:

#merged_data['slope'] = merged_data.groupby('Code').diff()(lambda v: linregress(v.Date, v.cases)[0])
#merged_data['slope'] = (merged_data.groupby('Code')['cases'].apply(lambda x: x.shift().expanding().mean()))
#merged_data['slope'] = (merged_data.groupby('Code')['Date','cases'].apply(lambda x,y : x.shift().expanding().apply(lambda v: linregress([0], [1])[0])))

In [ ]:
#jupyter nbconvert merge_data_modified.ipynb --to html --output covidCulture.html